# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

suma = 0

for sqrt_value in generator:
    suma = suma + sqrt_value


print(suma)


8.382332347441762


In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

values = list(generator)


values[12]

3.605551275463989

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [11]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [9]:
import dlt

In [17]:
pipeline = dlt.pipeline(pipeline_name="person_data",
						destination='duckdb',
						dataset_name='people_data')

In [18]:
person1 = list(people_1())

In [19]:
person1

[{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'},
 {'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'},
 {'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'},
 {'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'},
 {'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}]

In [20]:
# run the pipeline with default settings, and capture the outcome
info = pipeline.run(person1,
                    table_name="people",
                    write_disposition="replace")

# show the outcome
print(info)

Pipeline person_data load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////content/person_data.duckdb location to store data
Load package 1708301685.1160665 is LOADED and contains no failed jobs


In [25]:
import duckdb

conn = duckdb.connect("person_data.duckdb")
conn.sql(f"SET search_path = 'people_data' ")

In [26]:
display(conn.sql("show tables"))

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [29]:
display(conn.sql("SELECT SUM(age) FROM people"))

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [30]:
person2 = list(people_2())

In [31]:
pipeline.run(person2,
            table_name="people",
            write_disposition="append")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7d5ce2c3b550>, metrics={'1708303445.46285': [{'started_at': DateTime(2024, 2, 19, 0, 44, 5, 854188, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 19, 0, 44, 6, 106787, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////content/person_data.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people_data', loads_ids=['1708303445.46285'], load_packages=[LoadPackageInfo(load_id='1708303445.46285', package_path='/var/dlt/pipelines/person_data/load/loaded/1708303445.46285', state='loaded', schema=Schema person_data at 137837782230224, schema_update={'people': {'name': 'people', 'columns': {'occupation': {'name': 'occupation', 'data_type': 'text', 'nullable': True}}, 'write_disposition': 'append', 'resource': 'people', 'table_format': 

In [32]:
display(conn.sql("SELECT SUM(age) FROM people"))

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


<generator object people_1 at 0x7d5cdc0d37d0>

In [38]:
people_2()

In [33]:
pipeline = dlt.pipeline(pipeline_name="person_data2",
						destination='duckdb',
						dataset_name='people_data2')

In [40]:
pipeline.run(people_1(),
										table_name="people2",
										write_disposition="merge",
                    primary_key="ID")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7d5cdc0aaf50>, metrics={'1708304099.9660492': [{'started_at': DateTime(2024, 2, 19, 0, 55, 0, 385703, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 19, 0, 55, 1, 111714, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////content/person_data2.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people_data2', loads_ids=['1708304099.9660492'], load_packages=[LoadPackageInfo(load_id='1708304099.9660492', package_path='/var/dlt/pipelines/person_data2/load/loaded/1708304099.9660492', state='loaded', schema=Schema person_data2 at 137837782223984, schema_update={'_dlt_loads': {'name': '_dlt_loads', 'columns': {'load_id': {'name': 'load_id', 'data_type': 'text', 'nullable': False}, 'schema_name': {'name': 'schema_name', 'data_type': 'te

In [41]:
pipeline.run(people_2(),
										table_name="people2",
										write_disposition="merge",
                    primary_key="ID")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7d5cdc0aaf50>, metrics={'1708304116.813182': [{'started_at': DateTime(2024, 2, 19, 0, 55, 17, 201580, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 19, 0, 55, 17, 832412, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////content/person_data2.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people_data2', loads_ids=['1708304116.813182'], load_packages=[LoadPackageInfo(load_id='1708304116.813182', package_path='/var/dlt/pipelines/person_data2/load/loaded/1708304116.813182', state='loaded', schema=Schema person_data2 at 137837795471568, schema_update={'people2': {'name': 'people2', 'columns': {'occupation': {'name': 'occupation', 'data_type': 'text', 'nullable': True}}, 'write_disposition': 'merge', 'resource': 'people2', 'tab

In [42]:
conn = duckdb.connect("person_data2.duckdb")
conn.sql(f"SET search_path = 'people_data2' ")

In [43]:
display(conn.sql("SELECT * FROM people2"))

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708304099.9660492 │ QICBV1N/oY1lyQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708304099.9660492 │ gwm7Iy4MndK7vw │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1708304116.813182  │ eEKIQEPZmXmEhg │ Job_5      │
│     8 │ Person_8 │    38 │ City_B  │ 1708304116.813182  │ 305+ajZCX5eI9g │ Job_8      │
│     7 │ Person_7 │    37 │ City_B  │ 1708304116.813182  │ PuG7RBZtzJ1RZA │ Job_7      │
│     4 │ Person_4 │    34 │ City_B  │ 1708304116.813182  │ mqFVKiGYhHaCmg │ Job_4      │
│     3 │ Person_3 │    33 │ City_B  │ 1708304116.813182  │ uzgd/Qlh47A5Zw │ Job_3      │
│     6 │ 

In [44]:
display(conn.sql("SELECT sum(age) FROM people2"))

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

# Solution: First make sure that the following modules are installed:

In [8]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX